# Transfer-based AutoAttack generation (Torch + RobustBench)

This notebook generates adversarial samples using AutoAttack (L2) against a RobustBench pretrained model,
and saves them to `data/adversarial_samples/autoattack_torch/` in NHWC format for downstream TF2 pipelines.

In [ ]:
import os
import yaml
import numpy as np

import torch
import torch.nn as nn

from robustbench.data import load_cifar10
from robustbench.utils import load_model

## install autoattack first.

from autoattack import AutoAttack

from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

In [ ]:
from source.utils import load_yaml

PATHS = load_yaml("./configs/paths.yaml")
EXP   = load_yaml("./configs/exp.yaml")

data_root       = PATHS["data_root"]
autoattack_out  = PATHS["autoattack_out"]

seed            = int(EXP["seed"]) # 42
aa_cfg          = EXP["autoattack"]
norm, eps, bs, rb_model_name, rb_threat_model = "L2", 0.5, 524, "Standard", "L2"

os.makedirs(autoattack_out, exist_ok=True)

In [ ]:
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_gpus = torch.cuda.device_count()

## RobustBench model and AutoAttack setup
We use a RobustBench pretrained `standard` model and run AutoAttack ($l_2$, $\epsilon = 0.5$)

In [ ]:
model = load_model(
    model_name=rb_model_name,
    dataset="cifar10",
    threat_model=rb_threat_model
)
model = model.to(device)
model.eval()

if num_gpus > 1:
    model = nn.DataParallel(model)

adversary = AutoAttack(
    model,
    norm=norm,
    eps=eps
)

## Generate adversarial examples for CIFAR-10 test (RobustBench)
This uses `robustbench.data.load_cifar10()` which returns tensors in NCHW.
Outputs are saved in NHWC for TF2.

In [ ]:
x_test, y_test = load_cifar10()   # NCHW
x_test = x_test.to(device)
y_test = y_test.to(device)

x_adv_test = adversary.run_standard_evaluation(x_test, y_test, bs=bs)

x_adv = x_adv_test.detach().cpu()
x_adv_nhwc = x_adv.permute(0, 2, 3, 1).contiguous().numpy().astype(np.float32)


In [ ]:
out_prefix = "data/adversarial_samples/autoattack_torch/"
np.save(out_prefix + "x_adv_test_std_l2_eps5", x_adv_nhwc)


## Outputs

Saved under: `data/adversarial_samples/autoattack_torch/`

- `x_adv_test_std_l2_eps5.npy`